In [ ]:
import metmhn.regularized_optimization as reg_opt
import metmhn.Utilityfunctions as utils

import pandas as pd
import scipy.optimize as opt
import jax.numpy as jnp
import numpy as np
import jax as jax
jax.config.update("jax_enable_x64", True)
import logging
logging.basicConfig(filename='example.log', filemode='w', level=logging.INFO, force=True)

In [ ]:
#mut_handle = "../data/paad/G12_PAADPANET_PM_z10_EventsAEP.csv"
#mut_handle = "../data/paad/G12_PAADPANET_PM_z10_Events_Mut50_Full.csv"
mut_handle = "../data/paad/G12_PAADPANET_PM_z10_Events_30and5_Full.csv"
annot_handle = "../data/paad/G12_PAADPANET_PM_z10_sampleSelection.csv"
#paads_only =  pd.read_csv("../data/paad/ptMap.csv")
#mut_handle = "../data/luad/G13_LUAD_PM_v2_Events_20and15_Full.csv"
#annot_handle = "../data/luad/G13_LUAD_PM_v2_sampleSelection_20and15.csv"
#annot_handle = "../data/luad/G13_LUAD_PM_z1_sampleSelection_20and8.csv"
#mut_handle = "../data/luad/G13_LUAD_PM_z1_Events_20and8_Full.csv"
#annot_handle = "../data/paad/G13_PAADPANET_PM_v2_sampleSelection_30and15.csv"
#mut_handle = "../data/paad/G13_PAADPANET_PM_v2_Events_30and15_Full.csv"

annot_data = pd.read_csv(annot_handle)
mut_data = pd.read_csv(mut_handle)
mut_data.rename(columns={"Unnamed: 0":"patientID"}, inplace = True)
dat = pd.merge(mut_data, annot_data.loc[:, ['patientID', 'metaStatus']], on=["patientID", "patientID"])

# Remove datapoints, that consist solely of NaNs
dat = dat.loc[dat.iloc[:,1:-3].isna().all(axis=1) == False, :]
dat = dat.loc[(dat.iloc[:,1:-3].sum(axis=1) > 0), :]
dat.columns

In [ ]:
start = 'P.Mut.KRAS'
stop = 'M.Mut.PTPRT'
mult = dat.set_index(["paired", "metaStatus"])
cleaned = mult.loc[zip(*[[0,0,0,1],["present", "absent", "isMetastasis", "isPaired"]]), start:stop]
cleaned = cleaned.sort_index()
cleaned.loc[(0, ["present", "isMetastasis"]), "Seeding"] = 1
cleaned.loc[(0, "absent"), "Seeding"] = 0
cleaned.loc[(1, "isPaired"), "Seeding"] = 1
dat_prim_nomet, dat_met_only, dat_prim_met, dat_coupled = utils.split_data(cleaned)
print(dat_prim_nomet.shape[0], dat_prim_met.shape[0], dat_coupled.shape[0], dat_met_only.shape[0])

In [ ]:
events = []
for elem in cleaned.columns.to_list()[::2]:
    iwas = elem.split(".")
    if len(iwas) > 2:
        if iwas[1]  == 'Mut':
            events.append(iwas[2] + " (M)")
        else:
            events.append(iwas[1] + " (" + iwas[2] + ")")
    elif len(iwas) == 2:
        events.append(iwas[1])
    else:
        events.append("Seeding")

In [ ]:
n = (cleaned.shape[1]-1)//2
n += 1
indep = utils.indep(jnp.vstack((dat_met_only, dat_prim_met, dat_prim_nomet)), dat_coupled)
start_params = np.concatenate((indep.flatten(), np.zeros(n), np.zeros(n)))

In [ ]:
n_mod = n-1
arr = dat_coupled * np.array([1,2]*n_mod+[1])
arr = arr @ (np.diag([1,0]*n_mod+[1]) + np.diag([1,0]*n_mod, -1))
counts = np.zeros((6, n))
for i in range(0,2*n,2):
    i_h = int(i/2)
    for j in range(1,4):
        counts[j-1, i_h] = np.count_nonzero(arr[:,i]==j)/dat_coupled.shape[0]
    counts[3, i_h] = np.sum(dat_prim_nomet[:, i], axis=0)/dat_prim_nomet.shape[0]
    counts[4, i_h] = (np.sum(dat_prim_met[:, i], axis=0))/dat_prim_met.shape[0]
    counts[5, i_h] = (np.sum(dat_met_only[:, i+1], axis=0))/dat_met_only.shape[0]

labels = [["Coupled ("+str(dat_coupled.shape[0])+")"]*3 +\
          ["NM ("+str(dat_prim_nomet.shape[0])+")"] +\
          ["EM-PT ("+str(dat_prim_met.shape[0])+")"] +\
          ["EM-MT ("+str(dat_met_only.shape[0])+")"],
          ["PT-Private", "MT-Private", "Shared"] + ["Present"]*3]
       
inds =  pd.MultiIndex.from_tuples(list(zip(*labels)))
counts = pd.DataFrame(np.around(counts, 2), columns=events, index=inds).T
counts

In [ ]:
#utils.cross_val(cleaned, jnp.linspace(1e-04, 1e-03, 5), 3, start_params, 0.8, n)

In [16]:
penal1 = 0.0004 #L1 penalty on off-diagonals
m_p_corr = 0.8
x = opt.minimize(reg_opt.log_lik, x0 = start_params, args = (dat_prim_nomet, dat_prim_met, dat_met_only, dat_coupled, penal1, m_p_corr), 
                method = "L-BFGS-B", jac = reg_opt.grad, options={"maxiter":10000, "disp":True, "ftol":1e-04})

In [ ]:
df2 = pd.DataFrame(x.x[:n**2].reshape((n, n)), columns=events, index=events)
theta = df2.copy()
df2["fd_effects"] = np.array(x.x[n**2:n*(n+1)])
df2["sd_effects"] = np.array(x.x[n*(n+1):])
df2.to_csv("../results/luad/paired_20_8_0011.csv")
df2

In [ ]:
utils.plot_theta(df2, .3)